<a href="https://colab.research.google.com/github/hokoro/Crawling/blob/main/coffenbeancrawing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 설치(20.08 코랩 기준)
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
# 한글 폰트 설치
!apt-get install -y fonts-nanum*

     |████████████████████████████████| 968 kB 9.8 MB/s 
     |████████████████████████████████| 359 kB 42.6 MB/s 
     |████████████████████████████████| 138 kB 48.7 MB/s 
     |████████████████████████████████| 3.6 MB 59.3 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 58 kB 4.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

In [ ]:
from bs4 import BeautifulSoup #파이썬 전용 크롤링 라이브러리
import urllib.request #url 관련 처리 라이브러리
import pandas as pd #pandas 데이터 관리 전용 라이브러리
import datetime #일정/시간 관련 라이브러리 
from selenium import webdriver #동적 크롤링을 위한 라이브러리 = selenium 
import time #시간 관련 라이브러리

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#- 리눅스 서버와 같이 GUI를 제공하지 않는 환경에서는 '--no-sandbox, --disable-gpu' 옵션도 추가해야함.


In [ ]:
def CoffeeBean_store(result): 
  CoffenBean_URL  = "https://www.coffeebeankorea.com/store/store.asp" #CoffenBean 주소
  wd = webdriver.Chrome('chromedriver', chrome_options=chrome_options)  #webdriver 실행 파일을 Path 로 넣어 실행
 
  for i in range(1,10): #370개 데이터 크롤링 
    wd.get(CoffenBean_URL) #webdriver 에서 URL 링크 를 매개변수로 하여 페이지 연결
    time.sleep(1) #웹페이지 연결할 동안 1초 대기
    try:
      wd.execute_script("StorePop2(%d)" %i) #자바스크립트 함수 호출하여 매장 정보 페이지 열기 
      time.sleep(1) #다시 1초 대기를 한다
      html = wd.page_source #자바스크립트 함수가 호출된 소스 코드를 저장
      soupCB = BeautifulSoup(html,'html.parser') #BeautifulSoup 을 통해 해당 html 파일 인식
      store_name_h2 = soupCB.select("div.store_txt > h2") #div.store_txt 파일에서 h2 태그를 인식
      store_name = store_name_h2[0].string #h2 태그의 첫번쨰 원소  = 가게 이름 을 저장
      print(store_name) 
      store_info  = soupCB.select("div.store_txt > table.store_table > tbody > tr > td") #가게 정보는 td 태그 에 저장
      store_address_list = list(store_info[2])  #td태그의 3번쨰 원소 = 리스트 를 만들어 두번쨰 정보를 저장 
      store_address = store_address_list[0] #주소 리스트 에 첫번쨰 원소 인 가게 주소를 저장
      store_phone = store_info[3].string #가게 전화 번호는 가게 주소 리스트 에 3번째 원소를 string 형식으로 저장
      result.append([store_name] + [store_address] + [store_phone]) #result 에 가게 정보를 저장 
    except:
      continue
    
  return 


                

In [ ]:
def main():
  result = []
  print("CoffenBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>>")
  CoffeeBean_store(result) #크롤링 함수 호출
  CB_tbl = pd.DataFrame(result, columns = ('store' , 'address' , 'phone')) #가게정보 데이터 프레임 선언
  CB_tbl.to_csv('/content/CoffenBean.csv' , encoding = 'cp949',mode = 'w', index = True) #Data_frame -> csv 형식으로 변환하여 저장 

In [ ]:
if __name__ == '__main__':
  main()

CoffenBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until
